In [1]:
! python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Import file

In [2]:
import spacy
from spacy.matcher import PhraseMatcher
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
import re


# --- Load project titles from file ---

# Path to file
file_path = "4_esparcimiento_project_titles_normalized.txt"

# Open the file, read each line, and clean up spaces or empty lines
with open(file_path, "r", encoding="utf-8") as f:
    project_titles = [line.strip() for line in f if line.strip()]

print(f"{len(project_titles)} project titles loaded.")
print("Example titles:")
for title in project_titles[:5]:
    print(" •", title)



111 project titles loaded.
Example titles:
 • mejoramiento de infraestructura deportiva en la asociacion de vivienda 24 de junio distrito de coronel gregorio albarracin lanchipa  tacna  tacna
 • construccion de espacio publico recreativo en la junta vecinal la concordia  distrito de coronel gregorio albarracin lanchipa  tacna  tacna
 • construccion de la espacio publico recreativo en la asociacion vivienda talleres micro artesanales tacna atmat distrito de coronel gregorio albarracin lanchipa  tacna  tacna
 • construccion de espacio publico recreativo recreaciononal en la junta vecinal 28 de agosto ii etapa distrito de coronel gregorio albarracin lanchipa  tacna  tacna
 • construccion de infraestructura comunal multiuso la asociacion de vivienda los arenales  asociacion de vivienda roca eterna vinani iii etapa distrito de coronel gregorio albarracin lanchipa  tacna  tacna


Rule based matching model

In [3]:

# Load Spanish language model
nlp = spacy.load("es_core_news_lg")

# Category name (for later labeling)
CATEGORY = "SERVICIOS DE MOVILIDAD URBANA A TRAVÉS DE PISTAS Y VEREDAS"

# Indicator terms manually defined for this category
keywords = [
    "espacio publico recreativo",
    "espacio publico recreativo",
    "espacios publicos recreativos",
    "espacios publicos recreativos",
    "infraestructura deportiva",
    "infraestructura deportiva",
    "infraestructura cultural",
    "infraestructura cultural",
    "recreacion",
    "recreacion",
    "parque",
    "parque",
    "plaza",
    "plaza",
    "deportivo",
    "deportivo",
    "comunal",
    "comunal",
    "local comunal",
    "local comunal",
    "cultural",
    "cultural",
    "recreativos",
    "recreativos",
    "servicio de recreacion",
    "servicio de recreacion",
    "areas verdes",
    "areas verdes",
    "espacio publico urbano",
    "espacio publico urbano"
]




# Initialize a PhraseMatcher (better than TokenMatcher for multiword expressions)
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

# Convert keyword phrases into spaCy Doc objects and add to the matcher
patterns = [nlp(term) for term in keywords]
matcher.add(CATEGORY, patterns)

Rule based matching testing

In [4]:

# --- Evaluate matcher ---
y_true = [1] * len(project_titles)   # all titles truly belong to the category
y_pred = []

for pre_title in project_titles:
    title = re.sub(r'[^\w\sáéíóúüñ]', ' ', pre_title)

    doc = nlp(title)
    matches = matcher(doc)
    y_pred.append(1 if matches else 0)

# --- Compute evaluation metrics ---
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)
cm = confusion_matrix(y_true, y_pred)

print(f"\n Evaluation Results for Category: {CATEGORY}")
print(f"Total titles: {len(project_titles)}")
print(f"Detected as positive: {sum(y_pred)}")
print(f"Missed (false negatives): {y_pred.count(0)}")
print(f"Recall:    {recall:.3f}")
print(f"F1 Score:  {f1:.3f}")
print("\nConfusion Matrix (rows=True labels, cols=Predictions):")
print(cm)

print("\nDetailed Classification Report:")
print(classification_report(y_true, y_pred, target_names=["Not Category", "Category"]))


 Evaluation Results for Category: SERVICIOS DE MOVILIDAD URBANA A TRAVÉS DE PISTAS Y VEREDAS
Total titles: 111
Detected as positive: 73
Missed (false negatives): 38
Recall:    0.658
F1 Score:  0.793

Confusion Matrix (rows=True labels, cols=Predictions):
[[ 0  0]
 [38 73]]

Detailed Classification Report:
              precision    recall  f1-score   support

Not Category       0.00      0.00      0.00         0
    Category       1.00      0.66      0.79       111

    accuracy                           0.66       111
   macro avg       0.50      0.33      0.40       111
weighted avg       1.00      0.66      0.79       111



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


print the titles that dont belong to the category

In [ ]:
# Print titles that were NOT matched (false negatives)
not_detected = [title for title, pred in zip(project_titles, y_pred) if pred == 0]

print("\n🔎 Projects NOT detected as belonging to the category:")
if not_detected:
    for t in not_detected:
        print(" •", t)
else:
    print("✅ All projects were correctly detected by the matcher.")



🔎 Projects NOT detected as belonging to the category:
 • instalacion del sistema de alcantarillado en el barrio de incaraccay del centro poblado de patibamba del distrito de san miguel - provincia de la mar-ayacucho
 • ampliacion de redes de agua potable y alcantarillado en la asociacion de vivienda nueva esperanza, distrito de coronel gregorio albarracin lanchipa - tacna - tacna
 • construccion del sistema de agua potable y alcantarillado centro poblado sigual , distrito de paramonga - barranca - lima
 • mejoramiento de agua potable y alcantarillado lampay, distrito de paramonga - barranca - lima
 • instalacion del sistema de agua potable y alcantarillado en la asociacion de vivienda taller intiorko, distrito de pocollay - tacna - tacna
 • mejoramiento y ampliacion del sistema de agua potable y saneamiento del c.p. tunan, distrito de paramonga - barranca - lima
 • mejoramiento de los servicios de agua potable y saneamiento en los anexos poquera y chulibaya, distrito de ilabaya - jorg

aaaverify normalization problem

In [ ]:
for title in project_titles:
    if "alcantarillado" in title:
        print([ord(ch) for ch in title])
        break

[105, 110, 115, 116, 97, 108, 97, 99, 105, 111, 110, 32, 100, 101, 108, 32, 115, 105, 115, 116, 101, 109, 97, 32, 100, 101, 32, 97, 108, 99, 97, 110, 116, 97, 114, 105, 108, 108, 97, 100, 111, 32, 101, 110, 32, 101, 108, 32, 98, 97, 114, 114, 105, 111, 32, 100, 101, 32, 105, 110, 99, 97, 114, 97, 99, 99, 97, 121, 32, 100, 101, 108, 32, 99, 101, 110, 116, 114, 111, 32, 112, 111, 98, 108, 97, 100, 111, 32, 100, 101, 32, 112, 97, 116, 105, 98, 97, 109, 98, 97, 32, 100, 101, 108, 32, 100, 105, 115, 116, 114, 105, 116, 111, 32, 100, 101, 32, 115, 97, 110, 32, 109, 105, 103, 117, 101, 108, 32, 45, 32, 112, 114, 111, 118, 105, 110, 99, 105, 97, 32, 100, 101, 32, 108, 97, 32, 109, 97, 114, 45, 97, 121, 97, 99, 117, 99, 104, 111]


observe correctly clasified

In [ ]:
# Print titles that were matched
detected = [title for title, pred in zip(project_titles, y_pred) if pred == 1]

print("\n🔎 Projects detected as belonging to the category:")
if detected:
    for t in detected:
        print(" •", t)


🔎 Projects detected as belonging to the category:
 • mejoramiento de los servicios de agua potable y desague de la urb. san patricio, distrito de paramonga - barranca - lima
 • mejoramiento y ampliacion de los servicios de agua potable y desague de la urb. el olivar, distrito de paramonga - barranca - lima
 • mejoramiento y ampliacion de los servicios de agua potable y desague de la urb. 7 de junio, distrito de paramonga - barranca - lima
 • mejoramiento y ampliacion de los servicios de agua potable y desague de la urb. los jardines, distrito de paramonga - barranca - lima
 • instalacion del servicio de agua potable y unidades basicas de saneamiento (ubs) en el sector ccaccapancca, distrito de chilcas - la mar - ayacucho
 • mejoramiento de los servicios de agua potable y desague de la urb. vende barato, distrito de paramonga - barranca - lima
 • mejoramiento de los servicios de agua potable y desague de las urb. tres puentes, micaela bastidas, san francisco, republica de canada, distr